In [260]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
tf.__version__

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


'2.0.0-alpha0'

In [261]:
from tensorflow.python.client import device_lib

"GPU" + device_lib.list_local_devices()[-1].physical_device_desc.split(",")[1]

'GPU name: Tesla P100-PCIE-16GB'

In [262]:
import glob # The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. 
import imageio # Imageio is a Python library that provides an easy interface to read and write a wide range of image data, including animated images, volumetric data, and scientific formats. 
%matplotlib inline 
import matplotlib.pyplot as plt 
import tensorflow.keras.layers as layers # Keras layers API
import time
from IPython import display # For displaying image

In [263]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
import PIL

In [264]:
path = "../input/img_align_celeba/img_align_celeba/"

In [ ]:
ls

In [ ]:
# def train_step(images):
#     noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
#     # take the image links and return a cropped image
#     new_images = []
#     for file_name in images:
#         new_pic = load_image( path + file_name)
#         new_images.append(new_pic)
    
#     images = np.array(new_images)
#     images = images.reshape(images.shape[0], 64, 64, 3).astype('float32') # puts each number in its own numpy array so instead of [1,2,3] gonna be [[1], [2], [3]]
#     images = (images) / 255 # normalize to [0,1]
#     # Normalize the images to [-1. 1] so if it was 0 --> -1 and if it was 255 --> 1


In [ ]:
# def load_image( infilename ) :
#     img = PIL.Image.open( infilename )
#     img = img.crop([25,65,153,193])
#     img = img.resize((64,64))
#     data = np.asarray( img, dtype="int32" )
#     return data

In [ ]:
load_image(path + "000001.jpg").shape

In [ ]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.crop_to_bounding_box(image=image, 
                                        offset_height=65,
                                        offset_width= 25,
                                        target_height=193-65,
                                        target_width= 153-25)
  image = tf.image.resize(image,[64,64])

  image = tf.cast(image, dtype=tf.dtypes.float32)/255  # normalize to [0,1] range

  return image

def load_and_preprocess_image(filename):
  image = tf.io.read_file(filename)
  return preprocess_image(image)

img = load_and_preprocess_image(path + "000001.jpg")
plt.imshow(img)

In [ ]:
# clean up code image pipeline
"""
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.crop_to_bounding_box(image=image, 
                                        offset_height=65,
                                        offset_width= 25,
                                        target_height=193-65,
                                        target_width= 153-25)
  image = tf.image.resize(image,[64,64])

  image = tf.cast(image, dtype=tf.dtypes.float32)/255  # normalize to [0,1] range

  return image

def load_and_preprocess_image(filename):
  image = tf.io.read_file(filename)
  return preprocess_image(image)
  
image = tf.image.decode_jpeg(tf.io.read_file("../input/celeba-dataset/img_align_celeba/img_align_celeba/"+files[0]), channels=3)
image_c = tf.image.crop_to_bounding_box(image=image, 
                                        offset_height=65,
                                        offset_width= 25,
                                        target_height=193-65,
                                        target_width= 153-25)
plt.imshow(image_c)
"""

In [265]:
main_folder = '../input/'
data = pd.read_csv(main_folder + 'list_attr_celeba.csv')
data_2 = data.loc[:,['image_id', 'Young']]
data_2.head()
train_images = np.array(os.listdir(path))
data_shuffle = data_2.sample(frac = 1).reset_index(drop = True)
data_shuffle.replace(to_replace=-1, value=0, inplace=True)
data_shuffle.head()

BUFFER_SIZE = 20000 # number of images in training i think
BATCH_SIZE = 200 # This is just the standard number for batch size. Google for more info
# shuffle and batch the data
# np.random.shuffle(train_images)
train_images = np.split(data_shuffle['image_id'][:BUFFER_SIZE],BATCH_SIZE)
len(train_images)

200

In [266]:
data_shuffle.head()

,image_id,Young
0,096504.jpg,1
1,148862.jpg,1
2,186109.jpg,1
3,007346.jpg,1
4,040154.jpg,1


In [269]:
celeb_d = dict(zip(data_shuffle.image_id,data_shuffle.Young))

In [ ]:
celeb_d.keys()

In [ ]:
# https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
# In general, having all inputs to a neural network scaled to unit dimensions tries to convert the error surface into a more spherical shape. Hence, Gradient Descent converges faster, reducing training time.

In [ ]:
def make_generator_model():
  # In Keras, you assemble layers to build models. A model is (usually) a graph of layers. The most common type of model is a stack of layers
  model = tf.keras.Sequential()
  
  """
  Add a densely-connected layer to the model
  the model will take as input arrays of shape (*, 100).
  And and output arrays of shape (*, 7*7*256)
  
  after the first layer, you don't need to specify the size of the input anymore
  Afterwards, we do automatic shape inference
  """
  model.add(layers.Dense(4*4*1024, use_bias = False, input_shape = (100,)))
  
  """
  You can think about batch normalization as doing preprocessing at every layer of the network.
  Accelerating Deep Network Training by Reducing Internal Covariate Shift
  """
  model.add(layers.BatchNormalization())
  
  """
  ReLU is linear (identity) for all positive values, and zero for all negative values. 
  Leaky ReLU has a small slope for negative values, instead of altogether zero. For example, leaky ReLU may have y = 0.01x when x < 0
  """
  model.add(layers.LeakyReLU())
  
  # reshape the output from something flattened to something with a shape of (7,7,256)
  model.add(layers.Reshape(( 4, 4, 1024)))
  assert model.output_shape == (None, 4, 4, 1024) # Note: None is the batch size
  
  """
  The generator uses a transposed convolutional layer (Upsampling) layers to produce an image from seed (random noise).
  
  128 is the dimensionality of the output space
  (5,5) specifies the height and width of the 2D convolution window
  strides = (1,1) specifies the strides of the convolution along the height and width 
  
  """
  model.add(layers.Conv2DTranspose(512, (5, 5), strides = (2,2), padding = "same", use_bias = False))
  assert model.output_shape == (None, 8, 8, 512)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  
  # Another transposed convolutional layer (upsampling)
  model.add(layers.Conv2DTranspose(256, (5,5), strides = (2,2), padding = "same", use_bias = False))
  assert model.output_shape == (None, 16, 16, 256)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  
  # Another transposed convolutional layer (upsampling)
  model.add(layers.Conv2DTranspose(128, (5,5), strides = (2,2), padding = "same", use_bias = False))
  assert model.output_shape == (None, 32, 32, 128)
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  
  # Final output layer also a convolutional layer (upsampling), sigmoid goes from 0 to 1
  model.add(layers.Conv2DTranspose(3, (5,5), strides = (2,2), padding = "same", use_bias = False, activation = "sigmoid"))
  assert model.output_shape == (None, 64, 64, 3)
  
  
  return model

In [ ]:
class CelebPic(object):
    def __init__(self, path=None):
        self.path = path
        self.arr = None
        self.value = None

In [ ]:
celeb = CelebPic()

In [ ]:
celeb.path = "000001.jpg"

In [281]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', 
                                     input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3)) # Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.
      
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    """
    Flattens the input. Does not affect the batch size.
    If inputs are shaped (batch,) without a channel dimension, then flattening adds an extra channel dimension and output shapes are (batch, 1).
    """
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
     
    return model

In [309]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

tf.Tensor([[0.4994476]], shape=(1, 1), dtype=float32)


In [272]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

"""
Discriminator Loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predicitions on real images to an array of 1s
and the dicriminator's predicitons on fake (generated) images to an array of 0s.
"""
@tf.function
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  
  return total_loss

"""
Generator Loss

The generator's loss quantifies how well it was able to trick the discrimator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1).
Here, we will compare the discriminators decisions on the generated images to an array of 1s.
"""
@tf.function
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

"""
The discriminator and the generator optimizers are different since we will train two networks separately.
The Adam optimization algorithm is an extension to stochastic gradient descent.
Stochastic gradient descent maintains a single learning rate (termed alpha) for all weight updates and the learning rate does not change during training.
A learning rate is maintained for each network weight (parameter) and separately adapted as learning unfolds.

"""
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [273]:
checkpoint_dir = "../training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(
                                 discriminator_optimizer = discriminator_optimizer,
                                 discriminator = discriminator)

In [274]:
EPOCHS = 20
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier) to visualize progress in the animated GIF
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# new_images = []
# for file_name in train_images[0]:
#     new_pic = load_image( path + file_name)
#     #new_pic = tf.reshape(new_pic,[64,64,3])
#     new_images.append(new_pic)
    
# images = np.array(new_images)
# #images = tf.reshape(images.shape[0],[64,64,3])
# images = images.reshape(images.shape[0], 64, 64, 3).astype('float32') # puts each number in its own numpy array so instead of [1,2,3] gonna be [[1], [2], [3]]
# images = (images) / 255 # normalize to [0,1]

In [ ]:
# start = time.time()
# new_pic_tf = [tf.reshape(load_and_preprocess_image(path + file_name),[64,64,3]) for file_name in train_images[0]]
# images_tf = tf.convert_to_tensor(new_images_tf)
# print(time.time()-start)

In [ ]:
# start = time.time()
# new_images_tf = []
# for file_name in train_images[0]: #what is this train_images[0]
#     new_pic_tf = load_and_preprocess_image( path + file_name)
#     new_pic_tf = tf.reshape(new_pic_tf,[64,64,3])
#     new_images_tf.append(new_pic_tf)
# print(time.time()-start)

In [ ]:
[x for x in celeb_d if celeb_d[x]==1]

In [275]:
"""
The training loop begins with generator receiving a random seed as input. 
That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). 
The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.
"""

# Notice the use of tf.function
# This annotation causes the function to be "compiled"
def train_step(images): #images is the batch of filenames
    young = [x for x in images if celeb_d[x]==1]
    old = [x for x in images if celeb_d[x]==0]
    
    young_list = [tf.reshape(load_and_preprocess_image(path + file_name),[64,64,3]) for file_name in young]
    young_tf = tf.convert_to_tensor(young_list)
    
    old_list = [tf.reshape(load_and_preprocess_image(path + file_name),[64,64,3]) for file_name in old]
    old_tf = tf.convert_to_tensor(old_list)

    
    """
GradientTape() Records operations for automatic differentiation. Operations are recorded if 
they are executed within this context manager and at least one of their inputs is being "watched".
    """
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      young_output = discriminator(young_tf, training=True)
      old_output = discriminator(old_tf, training=True)

      disc_loss = discriminator_loss(young_output, old_output)
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    

In [276]:
#@tf.function
def train(dataset, epochs):  
  tf.print("Starting man!")
  for epoch in range(epochs):
    start = time.time()
    tf.print("Starting Epoch:", epoch)
    batch_count = 1
    for image_batch in dataset:
      tf.print("Batch:", batch_count)
      train_step(image_batch)
      print("Batch:", batch_count, "Complete")
      batch_count += 1
    

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    tf.print("Epoch:", epoch, "finished")
    tf.print()
    
    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    tf.print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    
  # Generate after the final epoch
  display.clear_output(wait=True)


In [277]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False. 
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False).numpy()

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i])
      plt.axis('off')
        
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [278]:
from IPython.display import Image
#Image(filename='image_at_epoch_0060.png') 

In [279]:
%%time
train(train_images, EPOCHS)

Epoch: 2 finished

Time for epoch 3 is 80.85759615898132 sec
Starting Epoch: 3
Batch: 1
Batch: 1 Complete
Batch: 2
Batch: 2 Complete
Batch: 3
Batch: 3 Complete
Batch: 4
Batch: 4 Complete
Batch: 5
Batch: 5 Complete
Batch: 6
Batch: 6 Complete
Batch: 7
Batch: 7 Complete
Batch: 8
Batch: 8 Complete
Batch: 9
Batch: 9 Complete
Batch: 10
Batch: 10 Complete
Batch: 11
Batch: 11 Complete
Batch: 12
Batch: 12 Complete
Batch: 13
Batch: 13 Complete
Batch: 14
Batch: 14 Complete
Batch: 15
Batch: 15 Complete
Batch: 16
Batch: 16 Complete
Batch: 17
Batch: 17 Complete
Batch: 18
Batch: 18 Complete
Batch: 19
Batch: 19 Complete
Batch: 20
Batch: 20 Complete
Batch: 21
Batch: 21 Complete
Batch: 22
Batch: 22 Complete
Batch: 23
Batch: 23 Complete
Batch: 24
Batch: 24 Complete
Batch: 25
Batch: 25 Complete
Batch: 26
Batch: 26 Complete
Batch: 27
Batch: 27 Complete
Batch: 28
Batch: 28 Complete
Batch: 29
Batch: 29 Complete
Batch: 30
Batch: 30 Complete
Batch: 31
Batch: 31 Complete
Batch: 32
Batch: 32 Complete
Batch: 33
B

KeyboardInterrupt: 

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
seed.shape[0]

In [ ]:
generator

In [ ]:
fig = plt.figure(figsize=(4,4))
predictions = generator(seed, training = False).numpy()
for i in range(seed.shape[0]):
    plt.subplot(4,4,i+1)
    plt.imshow(predictions[i], interpolation="nearest")
    plt.axis('off')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(4,4))
predictions = checkpoint.generator(seed, training = False).numpy()
for i in range(seed.shape[0]):
    plt.subplot(4,4,i+1)
    plt.imshow(predictions[i], interpolation="nearest")
    plt.axis('off')
    
plt.show()

In [ ]:
noise = tf.random.normal([1,100]) # shape is 1, 100
random_face = checkpoint.generator(noise, training = False).numpy()[0]
plt.imshow(random_face, interpolation='nearest')
plt.axis("off")
plt.show()